# from GSoC import splot

**A package providing lightweight plotting and mapping to facilitate spatial analysis with PySAL.**

The primary goal of this ['Google Summer of Code 2018' project](https://github.com/pysal/pysal/wiki/GSoC-2018---Geovisualization-Module-by-Stefanie-Lumnitz) was to design and implement the visualization package `splot`. [`splot`](https://github.com/pysal/splot) connects spatial analysis done in [`PySAL`](https://github.com/pysal) to visualization toolkits like [`matplotlib`](https://matplotlib.org). It provides users quick access to visualize popular PySAL objects, offering different views on their spatial analysis workflows. The `splot` package facilitates the creation of both static plots ready for publication and interactive visualizations for quick iteration and spatial data exploration. The project has successfully achieved its primary goal and `splot` was first released in the [PySAL 2.0 2rc](https://github.com/pysal/pysal/releases) on [July 19 2018](https://pypi.org/project/PySAL/2.0rc2/#files).

While developing `splot`, the potential integration of different popular visualization packages like [`bokeh`](https://bokeh.pydata.org/en/latest/) was explored. Based on the results API decisions were made, a cooperation with [`geopandas`](http://geopandas.org) was leveraged and the outline of the project changed slightly. The project now allows `splot` to potentially grow with interactive toolkits like `bokeh` in future, but focuses the current workflow on the creation of views with a `matplotlib` and `geopandas` backend. Additionally, developing new tools helped to assess PySAL's current code base and provided the opportunity to add or tweak functionality alongside `splot`'s visualizations to create an even more userfreindly library.

This notebook provides a summary of API decisions made, functionality developed and next steps planned towards a thriving `PySAL` and `splot` community in the context of the GSoC project.


## Contents

1. API Decisions
2. New `splot` functionality for:
    * esda
    * giddy
    * libpysal
    * mapping
3. Code beyoned splot
4. Remaining Work & Next Steps
5. Community Outreach

### GitHub and project links

* [GSoC GitHub project](https://github.com/pysal/splot/projects/1)
* [merged pull requests](https://github.com/pysal/splot/pulls?q=is%3Apr+author%3Aslumnitz+is%3Aclosed)
* [unmerged pull requests](https://github.com/pysal/splot/pulls/slumnitz)
* [Gitter channel](https://gitter.im/pysal-gsoc18/Lobby)
* [Project Blog](https://blogs.python-gsoc.org/stefanie-lumnitz/author/stefanie-lumnitz/)

## API Decisions

* [API Discussion](https://github.com/pysal/splot/issues/9)

## New splot functionality

* [Functionality support](https://github.com/pysal/splot/issues/10)

### splot.esda

### splot.giddy

### splot.libpysal

### splot.mapping

## Code beyoned splot

## Remaining work & next steps

## Community Outreach